# Introduction to evolutionary simulations with Geonomics (Part 2)

This is the second part of an introduction to evolutionary simulations with Geonomics. In this part, we will be running simulations of natural selection and then you will create and run your own simulations. As a reminder, *Geonomics* allows the user to test hypotheses and run simulations about how the spatial arrangement and size of populations of organisms while incorporating any realistic or modeled geospatial **layers** of a landscape. The so-called forces of evolution can all be simulated: mutation, natural selection, founder effects, genetic drift, and gene-flow. The result is a very flexible platform to allow prediction of evolutionary outcomes across space.

# Glossary

**A glossary of key terms and concepts is available here [Geonomics Glossary Google Doc](https://docs.google.com/document/d/133njF4NFe2SH5zSASTnObL5wGRp3RalnkFM6Aih-Wf0/edit?usp=sharing)




## Important notes: 

*Please don't be overwhelmed by the length of the code blocks in this notebook.* We intentionally left the code for you to see -- and exercises for you to modify the code -- so that you can think about the different parameters than need to be defined to construct a somewhat realistic evolutionary simulation.

You need to read the **text cells** carefully and follow the instructions. Some **code cells** you will *run without changing*. Other times you will need to *edit the cells as instructed* to adjust the parameters of the model so we can simulate different scenarios. Reading the green `# comments` in Geonomics code will help to understand what you are actually doing during model set-up,  running of the model, and model results evaluation, and for **creating your own scenario** at the end. It is not necessary for you to read all of the `# comments` or code during this Notebook, this is what a user of the Geonomics package will refer to while running their simulations. We will also make use of **functions** which are blocks of code that perform a task (you do not need to edit or make your own functions).

## Helpful Reminders: 

### Jupyter notebooks
When a cell is running the circle in the top right of the notebook next to Python 3 will be solid grey. Some cells take several seconds to run. Just be patient.


### Text cells
In a notebook, each rectangle containing text or code is called a *cell*.

Text cells (like this one) can be edited by double-clicking on them. They're written in a simple format called [Markdown](http://daringfireball.net/projects/markdown/syntax) to add formatting and section headings.  You don't need to learn Markdown, but you might want to.

After you edit a text cell, click the "run cell" button at the top to confirm any changes. (Try not to delete the instructions of the lab.)

### Code cells
Other cells contain code in the Python 3 language. Running a code cell will execute all of the code it contains.

To run the code in a code cell, first click on that cell to activate it.  It'll be highlighted with a little green or blue rectangle.  Next, either press the Run button or hold down the `shift` key and press `return` or `enter`.

The only code cells that need to be modified are right below a blue exercise box.

### Comments
Comments are statements in English that the computer ignores. We use comments to explain what the surrounding code does. Comments appear in green after the `#` symbol.

<div class="alert alert-info" role="alert" style="font-size:120%">
  
**Agenda**:
<ul>
    <li>1. run a simulation of natural selection to a heterogeneous environment</li>
    <li>2. create and run your own model scenario</li>
</ul>

<p>

</p>
</div>

### Okay! Now on to the code!

<div class="alert alert-warning" role="alert" style="font-size:120%">
    <b><h2>Import needed packages</h2></b> 

Run the following cell to bring in the necessary Python packages for our exercise today. Recall that you need to run this cell and any cells above the one you are working on each time you relaunch the notebook.

In [136]:
import geonomics as gnx
import numpy as np
import matplotlib.pyplot as plt
import nlmpy as nlmpy

# and also set Matplotlib's default plotting style and plot size
%matplotlib inline
plt.rcParams["figure.figsize"] = (9,4)

<div class="alert alert-warning" role="alert" style="font-size:120%">
    <b><h2>Define some helper functions</h2></b> 

Here we will define some functions to create simple, geometric arrays, which we'll use as landscapes for our simulations.

__No need to alter the code here.__ Just run it. We'll use the functions below.

In [137]:
################################
# RUN THIS CODE WITHOUT CHANGING
################################

def make_unif_array(n):
    """Makes a square array of ones, size n x n cells."""
    array = np.ones((n,n))
    return array

def add_vert_barrier(array, width = 5):
    """Returns a new array, which is the input array but with an added vertical barrier of the defined width."""
    if array.shape[0] % 2 != width % 2:
        width += 1
    assert width <= array.shape[0] - 2, ("The width of the barrier should be "
                                       "at least 2 less than the width of the landscape.")
    left_extent = int((array.shape[0] - width)/2)
    right_extent = left_extent + width
    new_array = np.copy(array)
    new_array[:, left_extent:right_extent] = 0
    return new_array

def make_horz_grad_array(n, grad_width=8):
    """Makes a square array with a horizontal gradient
    running from 0 to 1, size n x n cells."""
    assert grad_width <= n, "Argument 'grad_width' must be <= argument 'n'."
    if n % 2 != grad_width % 2:
        grad_width += 1
    grad_vals = np.linspace(0, 1, grad_width)
    append_len = int((n - grad_width) / 2)
    grad_vals = np.hstack([np.zeros((append_len)), grad_vals, np.ones((append_len))])
    array = np.vstack([grad_vals.T]*n)
    return array

# function for running and plotting genetic PCA
def plot_PCA(mod):
    from copy import deepcopy
    from sklearn.decomposition import PCA
    figsize = 6
    species = mod.comm[0]
    # get array of resulting genomic data (i.e. 'speciome'),
    # genotypes averaged by individual
    speciome = np.mean(species._get_genotypes(), axis=2)
    # run PCA on speciome
    pca = PCA(n_components=2)
    PCs = pca.fit_transform(speciome)
    # normalize the PC results
    norm_PCs = (PCs - np.min(PCs,
                             axis=0)) / (np.max(PCs,
                                                axis=0) - np.min(PCs,
                                                                 axis=0))
    # assign a value to each species, 0 or 1, indicating whether they're located on
    # the left or right vertical half of the landscape
    ind_colors = ['#00ffff' if ind.x < mod.land.dim[0]/2 else '#ff00ff' for ind in species.values()]
    # plot individuals on PCs 1 and 2, colored by their landscape half
    fig = plt.figure(figsize=(figsize, figsize), dpi= 80, facecolor='w', edgecolor='k')
    plt.scatter(norm_PCs[:,0], norm_PCs[:, 1], color = ind_colors)
    plt.xlabel('genetic PC 1')
    plt.ylabel('genetic PC 2')
    
# function for running and plotting genetic PCA
def map_PCA(mod, lyr_num=0, mask=True):
    from copy import deepcopy
    from sklearn.decomposition import PCA
    cmaps = {0: plt.cm.RdBu.copy(), 1: plt.cm.BrBG_r.copy()}
    mark_size = 60
    figsize = 8
    species = mod.comm[0]
    land = mod.land
    # get array of resulting genomic data (i.e. 'speciome'),
    # genotypes meaned by individual
    speciome = np.mean(species._get_genotypes(), axis=2)
    # run PCA on speciome
    pca = PCA(n_components=3)
    PCs = pca.fit_transform(speciome)
    # normalize the PC results
    norm_PCs = (PCs - np.min(PCs,
                             axis=0)) / (np.max(PCs,
                                                axis=0) - np.min(PCs,
                                                                 axis=0))
    # use first 3 PCs to get normalized values for R, G, & B colors
    PC_colors = norm_PCs * 255
    # scatter all individuals on top of landscape, colored by the
    # RBG colors developed from the first 3 geonmic PCs
    xs = mod.comm[0]._get_x()
    ys = mod.comm[0]._get_y()
    # get environmental raster, with barrier masked out
    env = deepcopy(mod.land[lyr_num].rast)
    if mask:
        env[mod.land[lyr_num].rast == 0] = np.nan
    # create light colormap for plotting landscape
    # bot = plt.cm.get_cmap('Blues', 256)(np.linspace(0.4, 0.45, 2))[0]
    # top = plt.cm.get_cmap('Reds', 256)(np.linspace(0.4, 0.45, 2))[0]
    # cols = np.vstack((top, bot))
    # cmap = mpl.colors.ListedColormap(cols, name='OrangeBlue')
    cmap = cmaps[lyr_num]
    cmap.set_bad(color='#8C8C8C')
    # plot landscape
    fig = plt.figure(figsize=(figsize, figsize), dpi= 80, facecolor='w', edgecolor='k')
    # plt.imshow(masked_env, cmap=cmap, alpha=0.8)
    plt.pcolormesh(land._x_cell_bds, land._y_cell_bds, env, cmap=cmap)
    # scatter plot of individuals, colored by composite PC score
    plt.scatter(xs, ys, c=PC_colors/255.0, s=mark_size, edgecolors='black')
    # fix x and y limits
    [f([dim - 0.5 for dim in (0, mod.land.dim[0])]) for f in (plt.xlim,
                                                              plt.ylim)]
    # get rid of x and y ticks
    [f([]) for f in (plt.xticks, plt.yticks)]



<div class="alert alert-danger" role="alert" style="font-size:120%">
    <b><h2>Part 1: Natural Selection</h2></b> 

**Natural selection occurs because**:
1. Individuals vary phenotypically.
2. The phenotypic variation has some genotypic underpinning.
3. Genotypic variations are heritable.
4. Variation among individuals makes some more likely to survive and reproduce than others (i.e. more fit).
5. More fit individuals pass on their genes more often.
6. Over time, this changes the genetic and phenotypic composition of a population and, thus, of a species.

Geonomics will simulate that process - in space! As you're watching these simulations, keep in mind the concepts you explored in the previous labs about gene flow and genetic drift. Those things still happen in this model, but natural selection adds an extra level of complexity. It can change the probability that certain genes move and persist on the landscape and, thus, the nature of isolation between different neighborhoods on the landscape.

The code in the following cells is very similar to what we saw in the first section. As before, run the code, changing any values as directed.


<div class="alert alert-warning" role="alert" style="font-size:120%">
    <b><h2>Set the parameter values</h2></b> 

This time, we'll create a model with <u>__two landscape layers__</u>:

1. A layer to describe the __habitat configuration__, which will determine both the species' local population density and its movement throughout the landscape. For the sake of simplicity, we will use a uniform layer for these simulations.
    
2. A layer to describe an __environmental gradient__ across the landscape, which will exert divergent selection on the species. We will use a linear gradient for these simulations.
    
We again want a model with a <u>__single species__</u>, but this time that species will have __one trait__. Each individual's phenotype for this trait will determined by its genotype at the genetic loci that underlie the trait, and its probability of survival will be determined by the difference between that phenotype and the optimal phenotype at the individual's location, according to the environmental gradient layer. Thus, the environmental gradient will exert divergent natural selection on this trait (selection for different phenotypes in different places).

We will simulate a single mouse species that will have a single trait: color. For the sake of simplicity, our mice have a single gene that controls color, which has three variations - white, green, and brown.  The second layer of our model defines the ground color across our landscape: one side is green, one side is brown, and the middle is white.  Mice survive best when their color matches the ground color. i.e. Green mice are favored on green ground, white mice are favored on white ground, and brown mice are favored on brown ground.  This is selective pressure.  In the simulation model we can define how strong this pressure is.  Those individuals that are best adapted to their location will survive longer and have more offspring, so the variations in the landscape are critical.


To accomplish all of that, we have changed the following parameter values (Reminder: you do not need to make any changes to the code, these definitions are to help you understand all the parameters of the model):

- **rast** parameter in the __'lyr_0'__ section: *We set this to a uniform raster, using __`make_unif_array(40)`__*.

- **rast** parameter in the __'lyr_1'__ section: *We set this to a horizontal gradient, using __`make_horz_grad_array(40)`__*

- **layer** parameter in the __'trait_0'__ section: *We set this to __`'lyr_1'`__, to indicate that layer 1 should serve as trait 0's selective force*


In [138]:
# selection_demo.py

# This is a parameters file generated by Geonomics
# (by the gnx.make_parameters_file() function).


                   #   ::::::          :::    :: :::::::::::#
             #::::::    ::::   :::      ::    :: :: ::::::::::: ::#
          #:::::::::     ::            ::   :::::::::::::::::::::::::#
        #::::::::::                      :::::::::: :::::: ::::::::  ::#
      #  : ::::  ::                    ::::  : ::    :::::::: : ::  :    #
     # GGGGG :EEEE: OOOOO   NN   NN   OOOOO   MM   MM IIIIII  CCCCC SSSSS #
    # GG     EE    OO   OO  NNN  NN  OO   OO  MM   MM   II   CC     SS     #
    # GG     EE   OO     OO NN N NN OO     OO MMM MMM   II   CC     SSSSSS #
    # GG GGG EEEE OO     OO NN  NNN OO     OO MM M MM   II   CC         SS #
    # GG   G EE    OO   OO  NN   NN  OO   OO  MM   MM   II   CC        SSS #
     # GGGGG :EEEE: OOOOO   NN   NN   OOOOO   MM   MM IIIIII  CCCCC SSSSS #
      #    : ::::::::               :::::::::: ::              ::  :   : #
        #:    :::::                    :::::: :::             :::::::  #
          #    :::                      :::::  ::              ::::: #
             #  ::                      ::::                      #
                   #                                        #
                      #  :: ::    :::             #


params = {
#--------------------------------------------------------------------------#

#-----------------#
#--- LANDSCAPE ---#
#-----------------#
    'landscape': {

    #------------#
    #--- main ---#
    #------------#
        'main': {
            #x,y (a.k.a. j,i) dimensions of the Landscape
            'dim':                      (40,40),
            #x,y resolution of the Landscape
            'res':                      (1,1),
            #x,y coords of upper-left corner of the Landscape
            'ulc':                      (0,0),
            #projection of the Landscape
            'prj':                      None,
            }, # <END> 'main'

    #--------------#
    #--- layers ---#
    #--------------#
        'layers': {

            #layer name (LAYER NAMES MUST BE UNIQUE!)
            'lyr_0': {

        #-------------------------------------#
        #--- layer num. 0: init parameters ---#
        #-------------------------------------#

                #initiating parameters for this layer
                'init': {

                    #parameters for a 'defined'-type Layer
                    'defined': {
                        #raster to use for the Layer
                        'rast':                   make_unif_array(40),
                        #point coordinates
                        'pts':                    None,
                        #point values
                        'vals':                   None,
                        #interpolation method {None, 'linear', 'cubic',
                        #'nearest'}
                        'interp_method':          None,

                        }, # <END> 'defined'

                    }, # <END> 'init'

                }, # <END> layer num. 0


            #layer name (LAYER NAMES MUST BE UNIQUE!)
            'lyr_1': {

        #-------------------------------------#
        #--- layer num. 1: init parameters ---#
        #-------------------------------------#

                #initiating parameters for this layer
                'init': {

                    #parameters for a 'defined'-type Layer
                    'defined': {
                        #raster to use for the Layer
                        'rast':                   make_horz_grad_array(40),
                        #point coordinates
                        'pts':                    None,
                        #point values
                        'vals':                   None,
                        #interpolation method {None, 'linear', 'cubic',
                        #'nearest'}
                        'interp_method':          None,

                        }, # <END> 'defined'

                    }, # <END> 'init'

                }, # <END> layer num. 1



    #### NOTE: Individual Layers' sections can be copy-and-pasted (and
    #### assigned distinct keys and names), to create additional Layers.


            } # <END> 'layers'

        }, # <END> 'landscape'


#-------------------------------------------------------------------------#
    
#-----------------#
#--- COMMUNITY ---#
#-----------------#
    'comm': {

        'species': {

            #species name (SPECIES NAMES MUST BE UNIQUE!)
            'spp_0': {

            #-----------------------------------#
            #--- spp num. 0: init parameters ---#
            #-----------------------------------#

                'init': {
                    #starting number of individs
                    'N':                250,
                    #carrying-capacity Layer name
                    'K_layer':          'lyr_0',
                    
###########################################################################
# CONSIDER CHANGING THIS PARAMETER                                        #
                    #multiplicative factor for carrying-capacity layer    #
                    'K_factor':         1,                                #
###########################################################################
                    
                    }, # <END> 'init'

            #-------------------------------------#
            #--- spp num. 0: mating parameters ---#
            #-------------------------------------#

                'mating'    : {
                    #age(s) at sexual maturity (if tuple, female first)
                    'repro_age':                0,
                    #whether to assign sexes
                    'sex':                      False,
                    #ratio of males to females
                    'sex_ratio':                1/1,
                    #whether P(birth) should be weighted by parental dist
                    'dist_weighted_birth':       False,
                    #intrinsic growth rate
                    'R':                        0.5,
                    #intrinsic birth rate (MUST BE 0<=b<=1)
                    'b':                        0.2,
                    #expectation of distr of n offspring per mating pair
                    'n_births_distr_lambda':    1,
                    #whether n births should be fixed at n_births_dist_lambda
                    'n_births_fixed':           True,
                    #whether individs should choose nearest neighs as mates
                    'choose_nearest_mate':        False,
                    #whether mate-choice should be inverse distance-weighted
                    'inverse_dist_mating':      False,

                    
########################################################
# CONSIDER CHANGING THIS PARAMETER                     #
                    #radius of mate-search area        #   
                    'mating_radius':            10,    #
########################################################            
                    
                    }, # <END> 'mating'

            #----------------------------------------#
            #--- spp num. 0: mortality parameters ---#
            #----------------------------------------#

                'mortality'     : {
                    #maximum age
                    'max_age':                      None,
                    #min P(death) (MUST BE 0<=d_min<=1)
                    'd_min':                        0,
                    #max P(death) (MUST BE 0<=d_max<=1)
                    'd_max':                        1,
                    #width of window used to estimate local pop density
                    'density_grid_window_width':    None,
                    }, # <END> 'mortality'

            #---------------------------------------#
            #--- spp num. 0: movement parameters ---#
            #---------------------------------------#

                'movement': {
                    #whether or not the species is mobile
                    'move':                                 True,
                    #mode of distr of movement direction
                    'direction_distr_mu':                   0,
                    #concentration of distr of movement direction
                    'direction_distr_kappa':                0,
##################################################################
# CONSIDER CHANGING THIS PARAMETER                               #
                    #1st param of distr of movement distance     #   
                    'movement_distance_distr_param1':       0.5, #
##################################################################
                    #2nd param of distr of movement distance
                    'movement_distance_distr_param2':       5e-8,
                    #movement distance distr to use ('levy' or 'wald')
                    'movement_distance_distr':              'levy',
                    #1st param of distr of dispersal distance
                    'dispersal_distance_distr_param1':      0,
                    #2nd param of distr of dispersal distance
                    'dispersal_distance_distr_param2':      5e-14,
                    #dispersal distance distr to use ('levy' or 'wald')
                    'dispersal_distance_distr':             'levy',
                    'move_surf'     : {
                        #move-surf Layer name
                        'layer':                'lyr_0',
                        #whether to use mixture distrs
                        'mixture':              True,
                        #concentration of distrs
                        'vm_distr_kappa':       12,
                        #length of approximation vectors for distrs
                        'approx_len':           5000,
                        }, # <END> 'move_surf'

                    },    # <END> 'movement'


            #---------------------------------------------------#
            #--- spp num. 0: genomic architecture parameters ---#
            #---------------------------------------------------#

                'gen_arch': {
                    #file defining custom genomic arch
                    'gen_arch_file':            None,
                    #num of loci
                    'L':                        100,
                    #starting allele frequency (None to draw freqs randomly)
                    'start_p_fixed':            0.5,
                    #whether to start neutral locus freqs at 0
                    'start_neut_zero':          False,
                    #genome-wide per-base neutral mut rate (0 to disable)
                    'mu_neut':                  0,
                    #genome-wide per-base deleterious mut rate (0 to disable)
                    'mu_delet':                 0,
                    #shape of distr of deleterious effect sizes
                    'delet_alpha_distr_shape':  0.2,
                    #scale of distr of deleterious effect sizes
                    'delet_alpha_distr_scale':  0.2,
                    #alpha of distr of recomb rates
                    'r_distr_alpha':            None,
                    #beta of distr of recomb rates
                    'r_distr_beta':             None,
                    #whether loci should be dominant (for allele '1')
                    'dom':                      False,
                    #whether to allow pleiotropy
                    'pleiotropy':               False,
                    #custom fn for drawing recomb rates
                    'recomb_rate_custom_fn':    None,
                    #number of recomb paths to hold in memory
                    'n_recomb_paths_mem':       int(1e4),
                    #total number of recomb paths to simulate
                    'n_recomb_paths_tot':       int(1e5),
                    #num of crossing-over events (i.e. recombs) to simulate
                    'n_recomb_sims':            10_000,
                    #whether to generate recombination paths at each timestep
                    'allow_ad_hoc_recomb':      False,
                    #whether to jitter recomb bps, to correctly track num_trees
                    'jitter_breakpoints':       False,
                    #whether to save mutation logs
                    'mut_log':                  False,
                    #whether to use tskit (to record full spatial pedigree)
                    'use_tskit':                True,
                    #time step interval for simplication of tskit tables
                    'tskit_simp_interval':      100,



                    'traits': {

                        #--------------------------#
                        #--- trait 0 parameters ---#
                        #--------------------------#
                        #trait name (TRAIT NAMES MUST BE UNIQUE!)
                        'trait_0': {
                            #trait-selection Layer name
                            'layer':                'lyr_1',
#################################################################
# CONSIDER CHANGING THESE PARAMETERS                            #
                            #polygenic selection coefficient    #
                            'phi':                  0.05,       # 
                            #number of loci underlying trait    #
                            'n_loci':               1,          #
#################################################################
                            #mutation rate at loci underlying trait
                            'mu':                   0,
                            #mean of distr of effect sizes
                            'alpha_distr_mu' :      0.1,
                            #variance of distr of effect size
                            'alpha_distr_sigma':    0,
                            #max allowed magnitude for an alpha value
                            'max_alpha_mag':        None,
                            #curvature of fitness function
                            'gamma':                1,
                            #whether the trait is universally advantageous
                            'univ_adv':             False
                            }, # <END> trait 0


    #### NOTE: Individual Traits' sections can be copy-and-pasted (and
    #### assigned distinct keys and names), to create additional Traits.


                        }, # <END> 'traits'

                    }, # <END> 'gen_arch'


                }, # <END> spp num. 0



    #### NOTE: individual Species' sections can be copy-and-pasted (and
    #### assigned distinct keys and names), to create additional Species.


            }, # <END> 'species'

        }, # <END> 'comm'


#------------------------------------------------------------------------#

#-------------#
#--- MODEL ---#
#-------------#
    'model': {
        #total Model runtime (in timesteps)
        'T':            100,
        #min burn-in runtime (in timesteps)
        'burn_T':       30,
        #seed number
        'num':          None,


        #-----------------------------#
        #--- iterations parameters ---#
        #-----------------------------#
        'its': {
            #num iterations
            'n_its':            1,
            #whether to randomize Landscape each iteration
            'rand_landscape':   False,
            #whether to randomize Community each iteration
            'rand_comm':        False,
            #whether to randomize GenomicArchitectures each iteration
            'rand_genarch':     True,
            #whether to burn in each iteration
            'repeat_burn':      False,
            }, # <END> 'iterations'



        } # <END> 'model'

    } # <END> params


<div class="alert alert-warning" role="alert" style="font-size:120%">
    <b><h2>Make the model</h2></b> 

Run the following code again to make the model.

In [ ]:
#make our params dict into a proper Geonomics ParamsDict object
params = gnx.make_params_dict(params, 'selection_demo')
#then use it to make a model
mod = gnx.make_model(parameters=params, verbose=True)

Then take a look at the model.

In [ ]:
print(mod)
%matplotlib inline
mod.plot(spp=0, lyr=1)

As a reminder, each point is an individual. The background is the environmental gradient that will exert divergent natural selection on the species.

<div class="alert alert-warning" role="alert" style="font-size:120%">
    <b><h2>Run the model</h2></b> 

Now we'll run the burn-in phase of the simulation, and then see what we've got:

In [141]:
%%capture
mod.walk(T=10000, mode='burn')

In [ ]:
mod.plot(spp=0, lyr=1)

Before we start running the main section of the model, we can use a built-in Geonomics function to make a map of the individuals, colored by their phenotypes for the trait.

**The closer an individual's color is to its environmental background, the closer that individual is to the optimum phenotype for its location, and thus the higher the individual's fitness.**

In [ ]:
mod.plot_phenotype(spp=0, trt=0)

And let's also create our genetic relatedness plots. As a reminder, in the first plot (pink and blue points), each point is an individual plotted in PCA space colored by whether they are on the right (pink) or left (blue) side of the landscape and the closer the individuals are together on the plot, the more similar they are genetically. In the second plot (rainbow points), the individuals are shown plotted on the landscape and the more similar their colors are the more similar they are genetically. This is a way to visualize the genetic structure of the population.

In [ ]:
plot_PCA(mod)
map_PCA(mod, lyr_num=1, mask=False)

Now, let's run our model for 100 timesteps, then create our plots again, up until time step 300. Keep a close eye on all three plots, and try and spot the changes in mismatch and structure as you run the simulation.

In [ ]:
mod.walk(100)

Timestep 100 plots:

In [ ]:
mod.plot_phenotype(spp=0, trt=0)

In [ ]:
plot_PCA(mod)
map_PCA(mod, lyr_num=1, mask=False)

In [ ]:
mod.walk(100)

Timestep 200 plots:

In [ ]:
mod.plot_phenotype(spp=0, trt=0)

In [ ]:
plot_PCA(mod)
map_PCA(mod, lyr_num=1, mask=False)

In [ ]:
mod.walk(100)

Timestep 300 plots:

In [ ]:
#trt - trait (in this case, trait 0)
mod.plot_phenotype(spp=0, trt=0)

In [ ]:
plot_PCA(mod)
map_PCA(mod, lyr_num=1, mask=False)

<!-- BEGIN QUESTION -->

<div class="alert alert-block alert-warning">
    <b> Use the plots above to answer Question 1 </b>
    <br />    
</div>

<div class="alert alert-block alert-info">
    <b>EXERCISE:</b>
    <br />
    Tweak the model and rerun.
    Create your own scenario, by changing one of the following parameters</u> as you wish in the code chunk below and then run a new model. Note that <b>each of these parameters is also outlined by a comment box within the original parameter code above</b>.

    
<b>Parameters to tweak</b>:
- ***phi***: *This controls the strength of selection on the trait. It can be set to 0 <= value <= 1. 0 would be no selection and 1 would be very strong selection*

- ***n_loci***: *This controls the number of genetic loci underlying the trait's phenotype. For this simulation, it can be set to 1 <= value <= 100. * (Remember, most traits are not controlled by a single gene, so this can be an important factor in the real world)*

- ***K_factor***: *This controls the population density, by setting the number of individuals that can inhabit each cell. It can be set to any value > 0, technically, but values larger than 3 or 4 will probably require too much memory to run on this server, so be careful!*  Higher and lower population density can have a number of impacts- like ability to find mates, availability of resources, etc.  Founder effects are hugely important here as well.

- ***movement_distance_distr_param_1***: *This controls the average distance that an individual moves each time step (in units of cell-widths), and thus how mobile the species is. For this simulation, it can be set to any value > 0 (but very large values might cause problems! If you want to test high migration, changing this value to 2 should be high enough).*

- ***mating_radius***: *This controls the radius (expressed in cell-width units) within which an indivudal can choose a mate. For this simulation, it can be set to any value > 0.*

If you are comfortable in Python, you can use the params dictionary above to also toy with any other parameters. But please know that by doing so you may break the code and be unable to run your simulation.
</div>

<!-- BEGIN QUESTION -->

<div class="alert alert-block alert-warning">
    <b> _Before you run your model, answer Question 2. Then, run your model and answer Question 3. Repeat this process for Question 4. </b>
    <br />    
</div>

Change the value of the parameter you want to change in this cell:

In [154]:
# First, we will make a copy of our original params dictionary to modify
import copy
params2 = copy.deepcopy(params)
# to change selection strength (phi), change 0.05 to any value between 0 and 1
params2['comm']['species']['spp_0']['gen_arch']['traits']['trait_0']['phi'] = 0.05
# to change the number of loci (n_loci), change 1 to any value between 1 and 100
params2['comm']['species']['spp_0']['gen_arch']['traits']['trait_0']['n_loci'] = 1
# to change the carrying capacity factor (K), change 1 to any value greater than 0
params2['comm']['species']['spp_0']['init']['K_factor'] = 1
# to change the mean movement distance (movement_distance_distr_param1), change 0.5 to any value greater than 0
params2['comm']['species']['spp_0']['movement']['movement_distance_distr_param1'] = 0.5
# to change the mating radius (mating_radius), change 10 to any value greater than 0
params2['comm']['species']['spp_0']['mating']['mating_radius'] = 10


Run your model by running the cells below (note: you may get a message about "stty: 'standard input': Inappropriate ioctl for device", which you can ignore)

In [ ]:
%%capture
#make your params dict into a proper Geonomics ParamsDict object
params2 = gnx.make_params_dict(params2, 'custom_mod')
#then use it to make a model
mod = gnx.make_model(parameters=params2, verbose=True)
# burn in your model
mod.walk(T=10000, mode='burn')

In [ ]:
# walk your model 
mod.walk(300)
# plot results
mod.plot_phenotype(spp=0, trt=0)
plot_PCA(mod)
map_PCA(mod, lyr_num=1, mask=False)

### OPTIONAL CHALLENGE
If you have time and want to make things more interesting, you can test out changing simulation parameters on a landscape with more realistic structure generated using a Neutral Landscape Model (NLM). We can create an NLM layer with the `nlmpy.mpd()` function. You can change the level of autocorrelation of this landscape by changing the `h` argument to any value between 0 and 1 where higher values = more spatial autocorrelation.

In [ ]:
from nlmpy import nlmpy

# Let's make another copy of our original params dictionary to modify
params3 = copy.deepcopy(params)

# Then, we can change the landscape to a new landscape generated using a midpoint NLM
# Try changing the h value to see how it affects the landscape
# (you need to keep nRow/nCol the same as these are the landscape dimensions already set for this model)
nlm = nlmpy.mpd(nRow=40, nCol=40, h=0.5) 
params3['landscape']['layers']['lyr_1']['init']['defined']['rast'] = nlm

# Plot the landscape
plt.imshow(nlm, cmap='viridis')

First, run an intial model with no parameters (other than the landscape) changed to establish your baseline for comparison:

In [ ]:
%%capture
#make your params dict into a proper Geonomics ParamsDict object
params3 = gnx.make_params_dict(params3, 'custom_mod')
#then use it to make a model
mod = gnx.make_model(parameters=params3, verbose=True)
# burn in your model
mod.walk(T=10000, mode='burn')

In [ ]:
# walk your model
mod.walk(300)
# plot results
mod.plot_phenotype(spp=0, trt=0)
plot_PCA(mod)
map_PCA(mod, lyr_num=1, mask=False)

Then you can then change any of the parameters described above and see how they effect evolution across a landscape with more realistic landscape structure:

In [160]:
# to change selection strength (phi), change 0.05 to any value between 0 and 1
params3['comm']['species']['spp_0']['gen_arch']['traits']['trait_0']['phi'] = 0.05
# to change the number of loci (n_loci), change 1 to any value between 1 and 100
params3['comm']['species']['spp_0']['gen_arch']['traits']['trait_0']['n_loci'] = 1
# to change the carrying capacity factor (K), change 1 to any value greater than 0
params3['comm']['species']['spp_0']['init']['K_factor'] = 1
# to change the mean movement distance (movement_distance_distr_param1), change 0.5 to any value greater than 0
params3['comm']['species']['spp_0']['movement']['movement_distance_distr_param1'] = 0.5
# to change the mating radius (mating_radius), change 10 to any value greater than 0
params3['comm']['species']['spp_0']['mating']['mating_radius'] = 10
# to change the level of spatial autocorrelation, change the h value to any number between 0 and 1
params3['landscape']['layers']['lyr_1']['init']['defined']['rast'] = nlmpy.mpd(nRow=40, nCol=40, h=0.5) 

In [ ]:
%%capture
#make your params dict into a proper Geonomics ParamsDict object
params3 = gnx.make_params_dict(params3, 'custom_mod')
#then use it to make a model
mod = gnx.make_model(parameters=params3, verbose=True)
# burn in your model
mod.walk(T=10000, mode='burn')

In [ ]:
# walk your model
mod.walk(300)
# plot results
mod.plot_phenotype(spp=0, trt=0)
plot_PCA(mod)
map_PCA(mod, lyr_num=1, mask=False)

This notebook was developed by Drew Hart, Natalie Graham, Monica Wilkinson and Keeley Takimoto (2019) and contributed to by Anusha Bishop (2020)